# 1 - Sources des données

## Données tabulaires - SAE

## Web Scrapping - Doctolib

Pour avoir un **indice de la disponibilité des IVGs en France**, nous allons utiliser des données issues du site de prise de rendez-vous médicales le plus utilisé en France, **Doctolib**. Par département et spécialisation IVG, les données extraites sont le nombre de localisations différentes proposant un RDV IVG dans un délai de 14, 30 et 90 jours. Ces données sont accessibles à tous gratuitement en utilisant leur moteur de recherche et des filtres (par [ex](https://www.doctolib.fr/ivg-medicamenteuse/paris?availabilitiesBefore=90&regulationSector%5B%5D=CONTRACTED_1&regulationSector%5B%5D=CONTRACTED_1_WITH_EXTRA&regulationSector%5B%5D=CONTRACTED_1_WITH_OPTAM&regulationSector%5B%5D=CONTRACTED_1_WITH_OPTAM_CO&regulationSector%5B%5D=CONTRACTED_2&regulationSector%5B%5D=CONTRACTED_2_WITH_OPTAM&regulationSector%5B%5D=CONTRACTED_2_WITH_OPTAM_CO&regulationSector%5B%5D=CONTRACTED&regulationSector%5B%5D=CONTRACTED_WITH_EXTRA&regulationSector%5B%5D=ORGANIZATION_CONTRACTED)).

Pour la mise en oeuvre, le script est `scrapper_doctolib.py` qui crée un .csv à l'aboutissement. Le script s'appuie sur la librairie `Playwright` permet de faire du web scrapping sur des sites dynamiques (comme Doctolib qui utilise react et javascript) en utilisant des navigateurs "headless" dans l'arrière plan. Avant de finir sur cette librairie, d'autres options ont été tentées : `BeautifulSoup` (pour adapté au parsing des sites statiques) et `Selenium` (une autre librairie de "headless browsers"). Cependant, `Playwright` a était le plus agile pour la gestion du chargement de la page. De plus, `asyncio` est utilisé pour faire du parallélisme et réduire le temps de scrapping.

> Point législatif
> 
> [Source (CNIL) : ](https://www.cnil.fr/fr/focus-interet-legitime-collecte-par-moissonnage)

Exemple des données obtenues (format dataframe pandas) suite au scraping (environ une demi-heure) :

In [2]:
from scripts.compilation_donnees import *

In [3]:
def importer_tout():
    df_SAE = importer_SAE()
    df_dep = importer_departement()
    df_finess = importer_finess()
    df_drees = importer_drees()
    df_pauv = importer_pauv()

    return df_SAE, df_dep, df_finess, df_drees, df_pauv

def importer_locale(date):
    folder = './donnees/'
    suffix = '_' + date + ".csv"

    df_SAE = pd.read_csv(folder + "SAE" + suffix)
    df_dep = pd.read_csv(folder + "dep" + suffix)
    df_finess = pd.read_csv(folder + "finees" + suffix)
    df_drees = pd.read_csv(folder + "drees" + suffix)
    df_pauv = pd.read_csv(folder + "pauv" + suffix)

    return df_SAE, df_dep, df_finess, df_drees, df_pauv

def sauvegarde_locale():
    df_SAE, df_dep, df_finess, df_drees, df_pauv = importer_tout()
    df_SAE.to_csv(get_path("SAE"), index=False)
    df_dep.to_csv(get_path("dep"), index=False)
    df_finess.to_csv(get_path("finees"), index=False)
    df_drees.to_csv(get_path("drees"), index=False)
    df_pauv.to_csv(get_path("pauv"), index=False)

    return 1

In [4]:
df_SAE, df_dep, df_finess, df_drees, df_pauv = importer_locale("2025-12-12-10-24")

# 2 - Statistiques descriptives 


In [5]:
df_nbcentre = pd.crosstab(df_finess["14"], df_finess["19"])
pd.set_option('display.max_columns', None)
df_nbcentre = df_nbcentre[["Centre de santÃ© sexuelle", "Centre gratuit d'information de dÃ©pistage et de diagnostic"]]
df_nbcentre.head(5)
# à normaliser par nombre d'habitant, pour 1000 habitant (pr avoir un chiffre interprétable)


#Centre de santÃ© sexuelle
#Centre gratuit d'information de dÃ©pistage et de diagnostic

19,Centre de santÃ© sexuelle,Centre gratuit d'information de dÃ©pistage et de diagnostic
14,,
AIN,1,2
AISNE,5,3
ALLIER,3,3
ALPES DE HAUTE PROVENCE,0,0
ALPES MARITIMES,8,1


In [7]:
# faire la moyenne des délais dans la SAE + nombre de centre qui font des IVG par département 
df_SAE.head(200)
#moy_delais = df_SAE.groupby(‘zone_geo’)[‘Valeur'].mean().reset_index()


,FI,PRIS,IVG,IVGN_1,IVGME,IVG1214,CONV,IMG,rs,stj,stjr,cat,catr,dep,Annee
0,010000024,1.0,513.0,517.0,356.0,30.0,NaN,17.0,CH DE FLEYRIAT,13,1,355,PUB2,01,2023
1,010000032,1.0,87.0,97.0,48.0,3.0,2.0,0.0,CH BUGEY SUD,13,1,355,PUB2,01,2023
2,010000065,1.0,73.0,NaN,73.0,0.0,NaN,0.0,CH DE TREVOUX - MONTPENSIER,13,1,355,PUB2,01,2023
3,010005239,1.0,117.0,126.0,105.0,1.0,NaN,0.0,CH DU HAUT BUGEY - GEOVREISSET,14,1,355,PUB2,01,2023
4,020000162,1.0,421.0,425.0,326.0,33.0,1.0,11.0,CH SAINT-QUENTIN,13,1,355,PUB2,02,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,400000105,1.0,211.0,204.0,132.0,10.0,NaN,1.0,CENTRE HOSPITALIER DAX,13,1,355,PUB2,40,2023
196,400000139,1.0,325.0,328.0,222.0,21.0,3.0,9.0,CHI MONT DE MARSAN ET PAYS DES SOURCES,13,1,355,PUB2,40,2023
197,400015236,1.0,15.0,NaN,0.0,2.0,0.0,0.0,GCS DU MARSAN,89,3,697,PNL10,40,2023
198,400780342,1.0,23.0,NaN,5.0,3.0,NaN,0.0,CLINIQUE JEAN LE BON,95,2,365,PLU4,40,2023
